In [1]:
import torch
import random
import torch.nn.functional as F
from torch import nn

In [2]:
def transpose(m):
    return [[m[j][i] for j in range(len(m))] for i in range(len(m[0]))]

In [51]:
class Conv2D(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size=(2, 2), stride=1, padding=0):

        # Set the input and output channels
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        # Set the kernel size, stride, and padding
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        # Initialize weights with torch
        self.torch_filter = torch.randn(out_channels, in_channels, kernel_size[0], kernel_size[1])

        # Initialize the weights manually
        self.filter = [[[[x+kernel_size[0]*y for x in range(1,kernel_size[0]+1)] for y in range(0, kernel_size[1])] for _ in range(0, in_channels)] for l in range(0,out_channels)]
        self.filter = torch.tensor(self.filter, dtype=torch.float64)

        # Assertions
        assert len(self.filter) == self.torch_filter.shape[0]
        assert len(self.filter[0]) == self.torch_filter.shape[1]
        assert len(self.filter[0][0]) == self.torch_filter.shape[2]
        assert len(self.filter[0][0][0]) == self.torch_filter.shape[3]

    def forward(self, input_batch):
        b, c, h, w = input_batch.size()

        # Define output shape
        output_height = int((h - self.kernel_size[0] + 2 * self.padding)/ self.stride + 1)
        output_width = int((w - self.kernel_size[1] + 2 * self.padding)/ self.stride + 1)

        # Initialize output with torch
        output_tensor_torch = torch.zeros(b, self.out_channels, output_height, output_width)

        # Initialize output with zeros
        output_tensor = [[[[0 for x in range(output_width)] for y in range(output_height)] for o in range(0, self.out_channels)] for image in range(0, b)]
        output_tensor = torch.tensor(output_tensor, dtype=torch.float64)

        # Assertions
        assert len(output_tensor) == output_tensor_torch.shape[0]
        assert len(output_tensor[0]) == output_tensor_torch.shape[1]
        assert len(output_tensor[0][0]) == output_tensor_torch.shape[2]
        assert len(output_tensor[0][0][0]) == output_tensor_torch.shape[3]

        # Unfold Input
        unfolded = F.unfold(input_batch, (2,2))

        # Reshape Filter
        filter_reshaped = self.filter.view(self.filter.size(0), -1).T

        # Perform Convolution
        conv_output = unfolded.transpose(1,2).matmul(filter_reshaped).transpose(1,2)

        # Fold Back
        out = F.fold(conv_output, output_tensor.shape[2:4], (1,1))

        return out

In [52]:
# Parameters
output_channels = 2 # Also the number of Filters
batch_size = 16 # How many examples (images) per batch
color_channels = 3 # The depth of a single example (RGB)
input_width = 32 # Width of an image
input_height = 32 # Height of an image

# Initialize Network

# Initialize input randomly
# input_batch = torch.randn(4, 3, 3, 3) # (Batch Size, Input Channels, Input Height, Input Width)

# Initialize input simplistic example
input_batch = [[[[x+input_width*y for x in range(1,input_width+1)] for y in range(0, input_height)] for _ in range(0, color_channels)] for b in range(batch_size)]
input_batch = torch.tensor(input_batch, dtype=torch.float64)


conv = Conv2D(in_channels=input_batch.shape[1], out_channels=output_channels)

# Start Forward Pass
output_batch = conv(input_batch)

Input Shape = torch.Size([16, 3, 32, 32])
Filter Shape = torch.Size([2, 3, 2, 2])
Output Shape B4 = torch.Size([16, 2, 31, 31])


